## Problem Solving As Search

### Search Algorithms for Decision Problems

### Artificial Intelligence 1: Week 2, part 2

## This lecture
Recap:
- problem solving as search,
- Search landscapes
- Generate and Test as a common framework

Search methods maintain ordered lists
- that represent the working memory
- in this module we are looking at 'single member' algorithms  
  where we generate test one new solution at a time

This week: Uninformed ‘Blind” search: depth/breadth-first

Next Week:  Adding heuristic measures: A*, best-first, hill-climbing


## Recap 1: Problem solving == searching for the right solution
- Learning and problem solving can be seen as a search through a set of possible solutions or states.

- Set of candidate solutions + move operator =>landscape

- Sometimes we have quality measures to help guide  search
  - landscape with extra dimension for quality
  - Ideas of local and global optima
- Sometimes not (decision problems)
  - ‘needle in a haystack’ – nothing to guide search
  - e.g.,finding the unlock code for a combination lock
- **Constructive** Search: build up partial solutions,   
  until we reach necessary complexity 
- **Perturbative** Search: all solutions have the same complexity


## Recap 2: Classes of problems
- an actual optimisation/modelling/prediction problem to solve is an *instance* of a class of problems
  - e.g. a specific route to find
  - a specific data set to train and test a model on
- The type of problem defines:
  - what decisions a candidate solution must specify
  - what values those decisions can have (Bools / floats / integer "value_set")
- a test() / evaluate() / score() method uses:
  - the variables that specify a unique candidate solution
  - the variables/data that specify a unique problem instance
  - returns a quality score
  - if the score is a Boolean** we have a **Decision problem**

Note that machine learning toolkits like sklearn/tensorflow  organise the classes and methods differently.  
But this is just an implementation choice as they are only designed only solve modelling probems.

## Recap 3: Solution = node in search graph / tree
<img src = "figures/solution_as_node.png" style="float:right" width =45%>

Depending on your preferred coding paradigm ( and language)
you could either encode solutions

As a data type e.g. a struct in C
```
typedef struct {  
  int numbers[N];  
  int depth;  
  int quality;  
  } solution*
```     
Or as a class in OOP e.g. python  
``` 
class CandidateSolution:
        self.variable_values 
        self.quality
        self.depth
        self.meets_constraints
```        

## Recap 4: different move operators create different landscapes 

In [11]:
# then we can have many move operators without changing the class definition
def Increment(attempt: CandidateSolution, decision: int):
    """changes the value of a single decision by +1"""
    attempt.variable_values[decision] += 1


def AddTwo(attempt: CandidateSolution, variableToChange: int):
    """changes the value of a single decision by +1"""
    attempt.variable_values[decision] += 2


def swapValues(attempt: CandidateSolution, decision1: int, decision2: int):
    """swaps the values of two decisions"""
    tmp = attempt.variable_values[decision1]
    attempt.variable_values[decision1] = attempt.variable_values[decision2]
    attempt.variable_values[decision2] = tmp

## Recap 5: Properties of Search Algorithms

Ways of generating solutions would ideally be:
- Optimal 
- Complete 
- Efficient 

Can't be all three, 
 - so **you** (the designer) have to make  a ** decision** about the best trade-offs
  - in the context of  **your** problem
  
<div> 
    <div style= "float:left" width=25%><img src="figures/optimal.png" width=200px></div>
     <div style= "float:left" width=25%><img src="figures/complete.png" width=200px> </div>
    <div  style="float:left" width=25%><img src="figures/efficient.png" width=200px></div>
</div>    





## Recap 6: Search using a Generate-test loop

- A common framework we can use to solve many different problems,
  - by changing the representation and  the test() function
- switching between:
  - different algorithms
    - by changing  how we specify SelectAndMoveFromOpenList() 
  - landscapes
    - by changing the ApplyMoveOperator() function
  - Constructive or Perturbative Search
    - constructive: ApplyMoveOperator()  **extends** working_candidate to make neighbours
    - perturbative: ApplyMoveOperator()  **changes values in** working_candidate to make neighbours

![flow diagram](./figures/generate-and-test.png)

## Quiz Questions:
- A point that is locally optimal for one landscape, will still be if you change the move operator? [True: False]
- In which if these situations might optimality be less important than efficiency?
  - Speech recognition software for dictation
  - Fingerprint recognition 
  - Neither
  - Both

- Is Exhaustive Search Optimal, Complete and Efficient (True: False x 3]



# Decision Problems and Uninformed search

- Some problems come with a natural measure of quality

- But sometimes we just have a ‘yes/no’ response:
  - Password cracking
  - ‘can I get from A to B’ without using toll roads?
  - Did I save that file ...
  


## Example: fox-grain-chicken

You have a fox, a chicken and a sack of grain. 

You must cross a river with only one of them at a time. 

- If you leave the fox with the chicken he will eat it; 

- If you leave the chicken with the grain she will eat it. 

Can you get all three across safely in less than N (e.g. ten)  moves?

## What type of problem is this?

Relevant "bit of the world"  is dictated by the rules:
they form the model of our system and the constraints

We are given “goal state” (final output to reach)

So this is an optimisation problem;
- Allowed moves defines a graph.
- The current state is defined by the position of the  fox, chicken, grain, and boat:  
  either  on first bank (0) or second bank (1)
- Seeking sequence of inputs that moves through graph from (0,0,0,0) to (1,1,1,1) 

**Constraints**: fox and chicken,  or chicken and grain can't be on same side without boat
 - i.e. solutions are **infeasible** if they pass through:
   -  {0,0,0,1},{1,1,1,0}   (both problem pairs left unattended)
   -  {0 0, 1,1}, {1,1,0,0}   (fox and chicken unattended)
   -  {0,1,1,0}, {1,0,0,1}  )chicken and grain unattended)


## Diagram of partial graph for this problem
Figure show partial graph for this problem, not all moves below final row shown. 
<img src = "figures/fox-chicken-grain-partial-graph.png" width=50%>

## How would you solve this?<img src = "figures/fox-chicken-grain-partial-graph.png" style = "float:right" width=25%>

Informally, if you give this to people as an exercise, what they do is:   
- start at one node of graph,
- follow one path e.g. {chicken,boat}->,  <-boat, ...  
  until they reach a problem (INFEASIBLE)   
  (either fox and chicken   
  or chicken and grain on the same side),
- then backtrack to previous “ok” node and try alternative move.

<div style="color:red;font-style:bold;font-size:20pt"> This is an example of Constructive Depth-First Search.</div>




# Depth-First Search for Decision Problems

Simple application of our single member search algorithm
- always pick the next item from the END of the open list
- this guarantees it is the last thing put on
- So we just treat the openlist like a **stack**

### Pseudocode within our framework
Only need to define one function
<div style="background:#F0FFFF">
    <dl>
        <dt><b>SelectAndMoveFromOpenList()</b></dt>
        <dd> my_index &larr; <b>GetLastIndex</b>(open_list)</dd>
        <dd> the_candidate &larr; open_list(my_index)</dd>
        <dd> <b>RemoveFromOpenList</b>(my_index)</dd>
        <dd> <b>Return</b>(the_candidate)</dd>
    </dl>
    </div>

## Depth-First Search Choices

**Constructive**: when
1. you don't know how complex the solution has to be  e.g., fox-chicken-grain type puzzles, tic-tac-toe, or
2. the constraints mean you can test and rule out unproductive branches before you get a complete solution e.g. NQueens

Potential large solutions means that you sometimes require *problem-specific code to detect loops* 
 

**Perturbative**:    when 
1. you know the complexity and  can only test complete solutions e.g. combination locking cracking, 
2. you can limit the complexity i.e. fox-grain-chicken  with only ten moves and 'do-nothing' added as a possible move

Really common idea is to think of the “atomic” move operator  
   i.e. the one that makes the smallest change
 

### What does this look like for fox-chicken-grain problem?

A candidate solution is a sequence of moves of boat with different (or no) cargo

We **decode** these to track the locations of (fox,chicken,grain,boat)  as the solution is played out
- a solution is invalid if it tries to move something that is not there
- or if it leaves the chicken alone with the fox or the grain

There are 8 moves in total {nothing,fox,chicken,grain} X {bank1to2, bank2to1}
- number these from 0 to 7: 



### So in our implementation:  

**FoxChickenGrain**.value_set = \[0,1,2,3,4,5,6,7]

**FoxChickenGrain**.evaluate(attempt:CandidateSolution): 
- initialises ```locations:dict = {'fox':0,'chicken':0,'grain':0,'boat':0}```
- loops through list ```attempt.variable_values```
   - **get** next move from list
   - **if**  move can't be applied (things in wrong places):   
       **return** INFEASIBLE (-1) 
  - **else**  
    apply move by editing ```locations```
  - **if** next state in forbidden list:  
        **return** INFEASIBLE (-1)
  - **if** next state = (1,1,1,1):    
      **return** SUCCESS (1)


### Choices for *ApplyMoveOperator()* 
- **perturbative** (assuming *fixed number of N* moves):  
   - nested loop through each position (1...N) and value (0...7) **changing**  a specific move to the new value
   - so each solution has *N* moves and 7*N* neighbours  
    (7 different values in *N* different position)  
  
- **constructive**:  
  - take working_candidate with n (<N) moves
  - loop through each possible extra move **adding** that to the *n* existing ones   
  - i.e.  each solution with *n* moves has  8 neighbours, all with *n+1* moves

## Depth-First Search Characteristics
**Efficient**:
- Can find solutions quickly.
- Only needs a small amount of storage space:
  - current solution, best seen, plus path followed. 

But not **Optimal** or **Complete**:
 - could get stuck for a long time searching an infinite or very deep branch of the tree,
 - especially if recursion is possible.
 - Hard to avoid this for constructive search.  
   - would have to write **problem-specific**  code that tracked what states are visited and flagged loops as infeasible
 -  If using a ‘perturbative’ approach can check whether solution has already been seen before adding it to open list

Implemented as a “stack” system: Last In First Out (LIFO)

## Breadth-First search<img src = "figures/fox-chicken-grain-partial-graph.png" style = "float:right" width=30%>
### Basic Idea
Examine all the possible options at each depth/level of the graph  
**before** proceeding to next level down the graph

In the context of **constructive** search this means:  
Examine all the solutions of a given complexity 
**before** increasing the complexity


## Breadth-First Search Pseudocode 
 
Only need to define one function.
Note only one line is different to depth-first search
<div style="background:#F0FFFF;">
    <dl>
        <dt><b>SelectAndMoveFromOpenList()</b></dt>
        <dd> my_index &larr; <b>GetFirstIndex</b>(open_list)</dd>
        <dd> the_candidate &larr; open_list(my_index)</dd>
        <dd> <b>RemoveFromOpenList</b>(my_index)</dd>
        <dd> <b>Return</b>(the_candidate)</dd>
    </dl>
    </div>

## Characteristics of Breadth-First Search 

Complete: Guaranteed to find solution if one exists.

Optimal: guaranteed to find closest solution to start

Efficient?
 - Works well when solution is near root,  
   especially if some branches are very deep. 
 - Higher Storage Overheads:  
   especially if branching factor at each node is high,  
   e.g. chess ….
   - have to store each node at current level.
 - have to store current tree – lots of retracing steps.

Implement as a Queue first-in-first-out (FIFO)

Often called “Flood-fill” in games/path-finding 
(because they like to think they’ve invented something)


## Depth-First vs Breadth First
### Example simple decision problem: Given the graph below is there a solution with cost <5?

<img src="figures/depth-with-list.png" width = 50%>
<img src="figures/breadth-with-list.png" width = 50%>



### Characteristics to take into account when choosing between them

- Depth-first is often quicker:
  - but may waste time in deep unproductive branches.
  - could apply a depth limit,  
    but then may never find solution.
- Depth-first will return first solution found
   – which may may not be the best.
- Breadth-first often slower, takes more storage, but is
  - “complete” i.e. guaranteed to find solution if one exists,
  - “optimal” i.e. will find the best solution at any given depth.
- Both are “tentative – they allow backtracking.
- Both can be applied to either constructive or perturbative search


## Quiz Questions
- Theseus in the maze with his ball of string, seeking the Minotaur, was doing?
- A search party fanning out and recruiting more people as they consider bigger areas is doing a parallel version of?

- which is which? black numbers show order nodes are examined, white numbers show the quality of that node

Which is which?
- X is often quicker
   - but may waste time in unproductive branches.
- X will return first solution found
    – that may not be the best / simplest.
    
- Y is often slower, 
- Y takes more storage, 
- but Y  is
  - “complete” i.e. guaranteed to find solution if one exists,
  - “optimal” i.e. will find the best solution at any given depth.

## Summary

Decision problems:    
- only a 'yes/no' answer
- can have multiple solutions with different complexity
- often associated with **Constraint Satisfaction Problems**

**Breadth-first** and **Depth-first**  are 'blind' or 'uninformed' search algorithms.  

You need to understand and be able to recognise:
 - when to apply them
 - what their characteristics are
 
If we give you a scenario you should be able to select an appropriate method and justify your choice.

### Next week:    search algorirthms for problem-solving guided by  a quality/cost function 
 